<a href="https://colab.research.google.com/github/fiorenza1116/Codeflix_Calculating_Churn/blob/main/Charging_Station_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electric Vehicle Charging Station Visual Analysis

Import Libraries

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import geopy.distance

Load Data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Alt Fuel Data
# Source:https://www.nyserda.ny.gov/All-Programs/Drive-Clean-Rebate/Charging-Options/Electric-Vehicle-Station-Locator#/find/nearest
fuel_stations = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Sources/alt_fuel_stations (Mar 11 2022).csv")

# Load Vehicle Population Data
# Source: https://www.atlasevhub.com/materials/state-ev-registration-data/
registered_evs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Sources/nj_ev_registrations_public.csv")

# Load Zip Code Data
# Source: https://www.unitedstateszipcodes.org/
zip_codes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Sources/zip_code_database.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6,15,16,20,33,34,43,49,52,57,58,59,60,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Organize Data 

In [ ]:
# Filter fuel_stations to only show electric stations in NJ
fuel_stations = fuel_stations[(fuel_stations['Fuel Type Code'] == 'ELEC') & (fuel_stations['State'] == 'NJ')]

# registered_evs contains only NJ registered vehicles already

In [ ]:
# Create a database and cursor for data transformation
con = sqlite3.connect('fuel_stations.db')

In [ ]:
# Merge register_evs with zip_codes to get access to latitude and longitude fields
registered_evs_merge = registered_evs.rename(columns={'ZIP Code' : 'zip_code', 'VIN Prefix' : 'vin'}).merge(zip_codes, left_on = 'zip_code', right_on = 'zip')[['vin', 'zip_code', 'latitude', 'longitude']]
registered_evs_merge.head()

,vin,zip_code,latitude,longitude
0,5YJYGDEE,7960,40.79,-74.47
1,5YJYGDEE,7960,40.79,-74.47
2,5YJXCDE2,7960,40.79,-74.47
3,5YJYGDEE,7960,40.79,-74.47
4,5YJXCDE2,7960,40.79,-74.47


In [ ]:
# Add file to created sql database
registered_evs_merge.to_sql(name = 'registered_evs', con = con, if_exists = 'replace')

In [ ]:
# Build table for density map in SQL
ev_density = pd.read_sql("SELECT zip_code, latitude, longitude, COUNT(vin) as EVs FROM registered_evs GROUP BY zip_code ORDER BY EVs DESC", con=con)
ev_density.head()

,zip_code,latitude,longitude,EVs
0,7677,41.02,-74.05,5433
1,8540,40.35,-74.65,4273
2,8831,40.34,-74.44,2286
3,8002,39.93,-75.03,2105
4,8820,40.58,-74.37,2052


In [ ]:
# Merge fuel_stations with zip for zip code latitude and longitude fields
# Rename columns to decipher where the lat/long fields are coming from
fuel_stations_center = fuel_stations.merge(zip_codes, left_on = 'ZIP', right_on = 'zip')[['Station Name', 'ZIP', 'Latitude', 'Longitude', 'latitude', 'longitude']]
fuel_stations_center = fuel_stations_center.rename(columns = {'Latitude' : 'latitude_fuel', 'Longitude' : 'longitude_fuel', 'latitude' : 'latitude_zip', 'longitude' : 'longitude_zip'})
fuel_stations_center.head()

,Station Name,ZIP,latitude_fuel,longitude_fuel,latitude_zip,longitude_zip
0,Element Hotels - Ewing,8628,40.286518,-74.807516,40.26,-74.83
1,New Jersey Coalition of Automotive Retailers,8628,40.252324,-74.840182,40.26,-74.83
2,GREENE 750 BUILDING NO. 11,8628,40.276372,-74.823103,40.26,-74.83
3,PALMER SQUARE CHAMBERS GARAGE,8542,40.350118,-74.662732,40.35,-74.66
4,SPRING ST. GAR SPRING STREET,8542,40.351268,-74.659425,40.35,-74.66


In [ ]:
# Using geopy to calculate the distance between the charging station and the zip code
def distance(latitude1, longitude1, latitude2, longitude2):
  location1 = (latitude1, longitude1)
  location2 = (latitude2, longitude2)

  try:
    return geopy.distance.geodesic(location1, location2).miles
  except ValueError:
    return np.nan
  
# Create avg_distance column by calling the distance function
fuel_stations_center['avg_distance'] = fuel_stations_center.apply(lambda x: distance(x['latitude_fuel'], x['longitude_fuel'], x['latitude_zip'], x['longitude_zip']), axis = 1)
fuel_stations_center.head()

,Station Name,ZIP,latitude_fuel,longitude_fuel,latitude_zip,longitude_zip,avg_distance
0,Element Hotels - Ewing,8628,40.286518,-74.807516,40.26,-74.83,2.181657
1,New Jersey Coalition of Automotive Retailers,8628,40.252324,-74.840182,40.26,-74.83,0.755119
2,GREENE 750 BUILDING NO. 11,8628,40.276372,-74.823103,40.26,-74.83,1.186978
3,PALMER SQUARE CHAMBERS GARAGE,8542,40.350118,-74.662732,40.35,-74.66,0.144450
4,SPRING ST. GAR SPRING STREET,8542,40.351268,-74.659425,40.35,-74.66,0.092605


Create Graph


In [ ]:
# Display Graphs
# Create Scatter Map Plot of Fuel Station Locations In New Jersey
fig = px.scatter_mapbox(fuel_stations, lat = 'Latitude', lon = 'Longitude', hover_name = 'Station Name', color_discrete_sequence = ['red'], zoom = 7.5, height = 850, width = 1000)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(margin = {'r': 0, 't': 0, 'l': 0, 'b': 0})

# Create density map of registered EVs in New Jersey
fig2 = px.density_mapbox(ev_density, lat = 'latitude', lon = 'longitude', z = 'EVs', radius = 40, mapbox_style = 'open-street-map', hover_name = "zip_code")
fig2.update_traces(opacity = 1, selector = dict(type = 'densitymapbox'))
fig2.update_traces(colorscale = [[0, 'rgb(40, 0, 0)'], [1, 'rgb(200, 0, 0)']], selector = dict(type = 'densitymapbox'))

fig.add_trace(fig2.data[0])

fig.show()
